# HCL App Security
Submitted by: </br>
Akash Nagraj (01FB15ECS047) </br>
Yash Mathur (01FB15ECS357)</br>
Mukund Sood (01FB15ECS363)</br>
Bishesh Sinha (01FB15ECS369)

In [241]:
#Keras Imports
import keras
from keras.optimizers import Adam
from keras.models import Model, Sequential
from keras.callbacks import ModelCheckpoint
from keras.preprocessing.text import Tokenizer
from keras.layers import Reshape, Flatten, Dropout, Concatenate
from keras.layers import Input, Dense, LSTM, Embedding, Conv1D, MaxPool1D

In [162]:
#Sklearn Imports
from sklearn import svm
from sklearn.metrics import accuracy_score
from sklearn.ensemble import VotingClassifier
from sklearn.datasets import make_classification
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

In [48]:
#Sklearn Imports
from xgboost import XGBClassifier

In [49]:
# GenSim Imports
from gensim.models import Word2Vec
from gensim.models import word2vec
from gensim.models.keyedvectors import KeyedVectors

In [50]:
# Miscellaneous Imports
import csv
import os
import re
import nltk
import numpy as np
import pandas as pd

In [51]:
# File paths to load:
dataset_path = "Processed.csv"
truth_value = "expectedresults-1.2.csv"

# Tunable Parameters

In [52]:
# Word Vector dimensions
vector_dimensions = 50

# Word2Vec model Training Window
window_size = 5

# Number of workers used to create the Word2Vec model
number_of_workers = 2

# Minimum number of times a word must be present to be
# included in the Word2Vec model
min_word_count = 2

# Specify name of the Word2Vec model to load
# If no name is specified, a model is built
Word2VecModelName = ""

# Number of epochs for Neural Network Training
training_epochs = 50

# Training batch size
training_batch_size  = 10

# Toggle normalization of labels: 1 -> Enable; 0 -> Disable
normalize_labels = 0

# Set the word limit for each code
max_length = 500

In [53]:
'''
Convert `raw_code` to a list of words:
'''
def code_to_wordlist(raw_code):
    '''
    The code block is converted to a list of words
    '''

    # Removing punctuation
    # table = string.maketrans('''!()-[]{};:'"\,<>./?@#$%^&*_~''')
    # raw_code.translate(table, string.punctuation)

    # Removing all numbers
    letters_only = re.sub("[^a-zA-Z]", " ", raw_code)

    # Converting all letters to lowercase
    words = letters_only.lower().strip().split()

    return words

In [54]:
'''
Convert `code` into lines
'''
def code_to_lines(code, tokenizer, remove_stopwords = False):
    '''
    The code is tokenized to form a list of sentences
    '''
    # Splitting each code block into lines
    raw_code = tokenizer.tokenize(code.strip())

    # Loop over each line of code
    lines = []
    for line in raw_code:

        # If a line of code is empty, skip it
        if(len(line) > 0):
            lines.append(code_to_wordlist(line))

    return lines

In [55]:
'''
Function to create word vectors
'''
def makeFeatureVectors(line, model, num_features):
    '''
    Create a word vectors of the words passed using the Word2Vec Model
    '''
    # Initial word vector to a zero vector
    featureVec = np.zeros((num_features,),dtype="float32")

    # Initialize Number of Words to 0
    nwords = 0
    
    # Index2word is a list that contains the names of the words in 
    # the model's vocabulary. Convert it to a set to improve speed
    index2word_set = set(model.wv.index2word)

    for word in line:
        if word in index2word_set:
            # Count the number of words
            nwords = nwords + 1
            # Create a vector by adding all word vectors it contains
            featureVec = np.add(featureVec,model[word])
    
    # Divide feature vector by 0 to get average word vector
    featureVec = np.divide(featureVec,nwords)

    return featureVec

In [56]:
'''
Function to create word vectors from a set of answers
'''
def getAvgFeatureVecs(code, model, num_features):
    '''
    Given a set of answers, calculate the average feature 
    vector and return a 2D numpy array for each one 
    '''
    # Initialize a counter
    counter = 0

    # Allocate a 2D numpy array, for speed
    codeFeatureVecs = np.zeros((len(code),num_features),dtype="float32")
    
    # Loop through the answers in an answer set
    for line in code:

       # Print a status message every 1000th answer
       if (counter%1000 == 0):
           print("CodeBlock %d of %d processed." % (counter, len(code)))
       
       # Get average feature vector for each answer in answer set
       codeFeatureVecs[counter] = makeFeatureVectors(line, model, num_features)
       
       # Increment the counter
       counter = counter + 1
        
    return codeFeatureVecs


In [10]:
def gen_labels (row):
        if row[" real vulnerability"] == True:
            return 1
        else:
            return 0
#     if row["Abbrev."] == "XSS":
#         return 1
#     elif row["Abbrev."] == "SECXSS2":
#         return 1
#     elif row["Abbrev."] == "SECCI" :
#         return 2
#     elif row["Abbrev."] == "SECSQLIJDBC":
#         return 3
#     elif row["Abbrev."] == "SECSQLISPRJDBC":
#         return 3
#     else: 
#         return 0
#     return 'Other'

In [57]:
def gen_category (row):
    if row["Abbrev."] == "XSS":
        return "xss"
    elif row["Abbrev."] == "SECXSS2":
        return "xss"
    elif row["Abbrev."] == "SECCI" :
        return "cmdi"
    elif row["Abbrev."] == "SECSQLIJDBC":
        return "sqli"
    elif row["Abbrev."] == "SECSQLISPRJDBC":
        return "sqli"
    elif row["Abbrev."] == "SECCU":
        return "securecookie"
    elif row["Abbrev."] == "SECHOC":
        return "securecookie"
    elif row["Abbrev."] == "SECPR":
        return "weakrand"
    elif row["Abbrev."] == "SECTBV":
        return "trustbound"
    elif row["Abbrev."] == "SECXPI":
        return "xpathi"
    elif row["Abbrev."] == "SECSHA1":
        return "hash"
    elif row["Abbrev."] == "SECPTI":
        return "pathtraver"
    elif row["Abbrev."] == "SECMD5":
        return "hash"
    elif row["Abbrev."] == "HRS":
        return "securecookie"
    elif row["Abbrev."] == "SECIC":
        return "securecookie"
    elif row["Abbrev."] == "SECLDAPI":
        return "ldapi"
    elif row["Abbrev."] == "SECDU":
        return "crypto"
    elif row["Abbrev."] == "CIPINT":
        return "hash" 
    elif row["Abbrev."] == "SECPTO":
        return "pathtraver"
    elif row["Abbrev."] == "PT":
        return "pathtraver"
    elif row["Abbrev."] == "STAIV":
        return "crypto"
    elif row["Abbrev."] == "PADORA":
        return "crypto"
    elif row["Abbrev."] == "SECSH":
        return "xss"
    elif row["Abbrev."] == "PADORA":
        return "crypto"
    elif row["Abbrev."] == "SECSP":
        return "xss"
    elif row["Abbrev."] == "SECSSQ":
        return "xss"
    elif row["Abbrev."] == "SECSP":
        return "xss"
    elif row["Abbrev."] == "SECHRS":
        return "xss"
    elif row["Abbrev."] == "SECSP":
        return "xss"
    elif row["Abbrev."] == "SECSHR":
        return "xss"
    elif row["Abbrev."] == "SQL":
        return "sqli"
    print("Error")

# Load the pre-computed vectors for the suspect lines

In [58]:
truth_values = pd.read_csv(truth_value, header = 0, delimiter = ",")
truth_values.rename(columns={'# test name': 'Filename',' category': 'category'}, inplace=True)
truth_values.head()

,Filename,category,real vulnerability,cwe,Benchmark version: 1.2,2016-06-1
0,BenchmarkTest00001,pathtraver,True,22,NaN,NaN
1,BenchmarkTest00002,pathtraver,True,22,NaN,NaN
2,BenchmarkTest00003,hash,True,328,NaN,NaN
3,BenchmarkTest00004,trustbound,True,501,NaN,NaN
4,BenchmarkTest00005,crypto,True,327,NaN,NaN


In [59]:
code = []
lines = []

In [60]:
column_headings = ["Filename", "Abbrev.", "Type", "Lineno.", "CodeBlock"]
dataset = pd.read_csv(dataset_path, header = 0, delimiter = ",", names = column_headings)
dataset.head()

,Filename,Abbrev.,Type,Lineno.,CodeBlock
0,BenchmarkTest01910,SECPR,PREDICTABLE_RANDOM,['53'],['double value = java.lang.Math.random();']
1,BenchmarkTest00577,SECCU,COOKIE_USAGE,"['91', '92', '113', '114']","['if (cookieName.equals(cookie.getName())) {',..."
2,BenchmarkTest00958,SECHOC,HTTPONLY_COOKIE,['36'],['javax.servlet.http.Cookie userCookie = new j...
3,BenchmarkTest00124,SECSH,SERVLET_HEADER,"['44', '45']","['if (request.getHeader(""BenchmarkTest00124"") ..."
4,BenchmarkTest00507,SECXSS2,XSS_SERVLET,"['92', '93', '94']","['response.getWriter().println(', 'user + "" ha..."


# Load the corresponding labels. The classes are defined as
0 - No Vulnerability <br>
1 - XSS <br>
2 - CLI <br>
3 - SLI <br>

In [61]:
dataset['category'] = dataset.apply(lambda row: gen_category(row),axis=1)
dataset.head()

,Filename,Abbrev.,Type,Lineno.,CodeBlock,category
0,BenchmarkTest01910,SECPR,PREDICTABLE_RANDOM,['53'],['double value = java.lang.Math.random();'],weakrand
1,BenchmarkTest00577,SECCU,COOKIE_USAGE,"['91', '92', '113', '114']","['if (cookieName.equals(cookie.getName())) {',...",securecookie
2,BenchmarkTest00958,SECHOC,HTTPONLY_COOKIE,['36'],['javax.servlet.http.Cookie userCookie = new j...,securecookie
3,BenchmarkTest00124,SECSH,SERVLET_HEADER,"['44', '45']","['if (request.getHeader(""BenchmarkTest00124"") ...",xss
4,BenchmarkTest00507,SECXSS2,XSS_SERVLET,"['92', '93', '94']","['response.getWriter().println(', 'user + "" ha...",xss


In [68]:
temp_df = pd.merge(dataset, truth_values, how='left')
temp_df.fillna("False", inplace=True)
temp_df['labels'] = temp_df.apply(lambda row: gen_labels(row),axis=1)
temp_df.head()

,Filename,Abbrev.,Type,Lineno.,CodeBlock,category,real vulnerability,cwe,Benchmark version: 1.2,2016-06-1,labels
0,BenchmarkTest01910,SECPR,PREDICTABLE_RANDOM,['53'],['double value = java.lang.Math.random();'],weakrand,True,330,False,False,1
1,BenchmarkTest00577,SECCU,COOKIE_USAGE,"['91', '92', '113', '114']","['if (cookieName.equals(cookie.getName())) {',...",securecookie,False,False,False,False,0
2,BenchmarkTest00958,SECHOC,HTTPONLY_COOKIE,['36'],['javax.servlet.http.Cookie userCookie = new j...,securecookie,False,False,False,False,0
3,BenchmarkTest00124,SECSH,SERVLET_HEADER,"['44', '45']","['if (request.getHeader(""BenchmarkTest00124"") ...",xss,False,False,False,False,0
4,BenchmarkTest00507,SECXSS2,XSS_SERVLET,"['92', '93', '94']","['response.getWriter().println(', 'user + "" ha...",xss,False,False,False,False,0


In [69]:
#Loading punkt tokenizer
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

#Loading keras tokenizer
t = Tokenizer()
t.fit_on_texts(temp_df["CodeBlock"])
word_index = t.word_index
vocab_size = len(t.word_index) + 1

In [70]:
for code_block_in_consideration in temp_df["CodeBlock"]:
    # Append the lines of code
    lines += code_to_lines(code_block_in_consideration, tokenizer, True)
    # Append the code blocks
    code.append(code_block_in_consideration)

In [71]:
# Check if a Word2Vec Model name is specified
if(Word2VecModelName):
    # Load a locally saved model
    v2wmodel = Word2Vec.load(Word2VecModelName)
else:
    # Building the Word2Vec Model with the specified parameters
    v2wmodel = word2vec.Word2Vec(lines, size=vector_dimensions, window=window_size, min_count=min_word_count, workers=number_of_workers)
    # Save Word2Vec model with specified Name
    v2wmodel.save("Word2VecModel")

In [26]:
blockTrainVectors = getAvgFeatureVecs(code, v2wmodel, vector_dimensions)

CodeBlock 0 of 6845 processed.
CodeBlock 1000 of 6845 processed.
CodeBlock 2000 of 6845 processed.
CodeBlock 3000 of 6845 processed.

/home/mathuryash5/anaconda3/envs/tf/lib/python3.5/site-packages/ipykernel_launcher.py:23: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).



CodeBlock 4000 of 6845 processed.
CodeBlock 5000 of 6845 processed.
CodeBlock 6000 of 6845 processed.


# Split the Dataset into Train set and Test Set

In [27]:
X_train, X_test, y_train, y_test = train_test_split(blockTrainVectors, temp_df["labels"], test_size=0.20, shuffle=True, random_state=42)

# Create a SVM classifier

In [113]:
# svm_clf = svm.SVC(decision_function_shape='ovo')
svm_clf = svm.SVC()

In [114]:
svm_clf.fit(X_train, y_train)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [115]:
predicted_svm_clf = svm_clf.predict(X_test)
accuracy_svm_clf = accuracy_score(y_test, predicted_svm_clf)
print("Accuracy: %.2f%%" % (accuracy_svm_clf * 100.0))

Accuracy: 71.44%


# SVM (One Vs Rest Classifier)

In [33]:
lin_svm_clf = svm.LinearSVC()

In [34]:
lin_svm_clf.fit(X_train, y_train)

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)

In [35]:
predicted_lin_svm_clf = lin_svm_clf.predict(X_test)
accuracy_lin_svm_clf = accuracy_score(y_test, predicted_lin_svm_clf)
print("Accuracy: %.2f%%" % (accuracy_lin_svm_clf * 100.0))

Accuracy: 72.10%


# Random Forrest Classifier

In [125]:
random_forrest_clf = RandomForestClassifier(max_depth=10, random_state=0)
random_forrest_clf.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=10, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=0, verbose=0, warm_start=False)

In [126]:
random_forest_pred = random_forrest_clf.predict(X_test)
# evaluate predictions
accuracy = accuracy_score(y_test, random_forest_pred)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 81.81%


# Xgboost

In [154]:
xgboost_model = XGBClassifier(max_depth=7, learning_rate=0.2, n_estimators=200)

In [159]:
xgboost_model.fit(X_train, y_train, verbose=True)
# make predictions for test data
y_pred = xgboost_model.predict(X_test)
xgboost_predictions = [round(value) for value in y_pred]

/home/mathuryash5/anaconda3/envs/tf/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [160]:
# evaluate predictions
accuracy = accuracy_score(y_test, xgboost_predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 81.88%


# Ensemble

In [255]:
eclf1 = VotingClassifier(estimators=[('xg', xgboost_model), ('rf', random_forrest_clf)], voting='hard')
eclf1 = eclf1.fit(X_train, y_train)

In [256]:
ensemble_pred = eclf1.predict(X_test)

/home/mathuryash5/anaconda3/envs/tf/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/mathuryash5/anaconda3/envs/tf/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [257]:
# evaluate predictions
accuracy = accuracy_score(y_test, ensemble_pred)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 81.45%


# CNN

In [223]:
sequence_length = blockTrainVectors.shape[1] 
embedding_dim = 128
filter_sizes = [3,4,5]
num_filters = 512
drop = 0.5

epochs = 100
batch_size = 30

In [249]:
model = Sequential()
model.add(Dense(32, input_dim=50, activation='relu'))
model.add(Conv1D(filters=32, kernel_size=2, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())
model.fit(X_train, y_train, epochs=3, batch_size=64)
# Final evaluation of the model
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

model.summary()

ValueError: Input 0 is incompatible with layer conv1d_6: expected ndim=3, found ndim=2

In [225]:
checkpoint = ModelCheckpoint('weights.{epoch:03d}-{val_acc:.4f}.hdf5', monitor='val_acc', verbose=1, save_best_only=True, mode='auto')
adam = Adam(lr=1e-4, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)

model.compile(optimizer=adam, loss='binary_crossentropy', metrics=['accuracy'])

In [226]:
model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, verbose=1, callbacks=[checkpoint], validation_data=(X_test, y_test))  # starts training

Train on 5476 samples, validate on 1369 samples
Epoch 1/100


InvalidArgumentError: indices[6,31] = -1 is not in [0, 2035)
	 [[Node: embedding_17/GatherV2 = GatherV2[Taxis=DT_INT32, Tindices=DT_INT32, Tparams=DT_FLOAT, _class=["loc:@training_2/Adam/gradients/embedding_17/GatherV2_grad/Reshape"], _device="/job:localhost/replica:0/task:0/device:CPU:0"](embedding_17/embeddings/read, embedding_17/Cast, embedding_17/GatherV2/axis)]]

Caused by op 'embedding_17/GatherV2', defined at:
  File "/home/mathuryash5/anaconda3/envs/tf/lib/python3.5/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/mathuryash5/anaconda3/envs/tf/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/mathuryash5/anaconda3/envs/tf/lib/python3.5/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/mathuryash5/anaconda3/envs/tf/lib/python3.5/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/mathuryash5/anaconda